## Steps 
### 1. process the wnut dataset 
### 2.make dataloader
### 3.train and evaluate

In [128]:
from datasets import load_dataset
wnut = load_dataset("wnut_17")
wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [129]:
#wnut['train']['ner_tags']

In [130]:
train=wnut['train']
#train[0]

### Step1.1: tokenise the dataset with the bert tokeniser 
why : bert doesnt know words it knows tokens , and our taks is named entity recognition ,the entitiew oudl be the tokens it knows not the words ,

### step1.2 break dataset into tokens with bert tokenizer

In [131]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [132]:
datatokens=tokenizer(train['tokens'], truncation=True, is_split_into_words=True)
#datatokens

### aligh tokens with the labels

In [133]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length", 
        max_length=128
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  
        label_ids = []
        previous_word = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != previous_word:
                label_ids.append(label[word_id])
            else:
                label_ids.append(-100)
            previous_word = word_id
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [134]:
tokenized_inputs=tokenize_and_align_labels(train)

In [135]:
tokenized_wnut = wnut.map( tokenize_and_align_labels, batched = True)


## step 2 we fix the padding 
:Pad dynamically at batch time
we will be adding padding to sequences of tokens to ensre all sequences in the batch has the same length

In [136]:
from transformers import DataCollatorForTokenClassification


In [137]:
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer) #for the padding thing in the training loop the padding is handled manually so this would be used for the evlaution


In [138]:
#!pip install -q evaluate seqeval


In [139]:
import evaluate
seqeval = evaluate.load("seqeval") #loads the SeqEval metric, which is a standard evaluation metric for sequence labeling task here is the NER

why we need to use this for evalaution metric?
the seqeval is the standard evlautation metric for sequence lableing tasks a7ki m3ehom
the seqeval compsed of your typical evlaution metriccs such as 
precision -recall-f1score-accurancy but the evalaution is donde at entity level , not tokenbytoken 
what does that mean?
since what we are doing is not prediction the label fro the token we created (since we tokenised the entire dataset) we want to predict for the entire entitiy (before tokenisation)
so the seqeval do entire entity level evalutation and 
since the process was taking on the first part of the word when we tokenised and the second part was represented by the -100 after we do our labeling (the -100 stay -100 dont get a label) then we reconstract the original world level prediction
 

In [140]:
import numpy as np


label_list = wnut["train"].features[f"ner_tags"].feature.names




In [141]:
example = wnut["train"][0]
labels = [label_list[i] for i in example[f"ner_tags"]]


In [142]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
}

## Training process

In [143]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

 the mapping of the ids of lables

In [144]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
    }
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [145]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [146]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [147]:
#train

In [148]:
num_labels = wnut['train'].features['ner_tags'].feature.num_classes
num_labels

13

In [149]:
train_dataset = tokenized_wnut["train"]
val_dataset = tokenized_wnut["validation"]
test_dataset = tokenized_wnut["test"]

now since we are working with a huggin face dataset lets make the dataloader and dataset

In [165]:
#make dataset
class NER_dataset(Dataset):
    def __init__(self,dataset):
        self.dataset=dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self,idx):
        item = self.dataset[idx]
        return {
            "input_ids": torch.tensor(item['input_ids']),
            "attention_mask": torch.tensor(item['attention_mask']),
            "labels": torch.tensor(item['labels'])
        }

In [166]:
train_dataset = NER_dataset(tokenized_wnut["train"])
val_dataset = NER_dataset(tokenized_wnut["validation"])


In [167]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True) #in trainng the padding is handled byt itself
data_collator = DataCollatorForTokenClassification(tokenizer)

val_loader = DataLoader(val_dataset, batch_size=16, collate_fn=data_collator)


now get the model

In [153]:
#we import the model
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels = 13,
    id2label = id2label,
    label2id= label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [154]:
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F


In [155]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

loss_function = CrossEntropyLoss(ignore_index=-100) #the left behind half wanna be token but not cool enaugh
optimizer = Adam(model.parameters(), lr=2e-5)

In [156]:
num_epochs=5

In [157]:
model.to(device)


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
   

In [158]:
next(model.parameters()).device

device(type='cuda', index=0)

In [164]:
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for input_ids, attention_mask, labels in train_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss  # or use loss_function(outputs.logits.view(-1, num_labels), labels.view(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1, Loss: 0.2959
Epoch 2, Loss: 0.1298
Epoch 3, Loss: 0.0757
Epoch 4, Loss: 0.0465
Epoch 5, Loss: 0.0304


In [168]:
def get_batch_accuracy(preds, labels):
    mask = labels != -100
    correct = (preds == labels) & mask
    return correct.sum().item() / mask.sum().item()


In [169]:
model.eval()
val_loss = 0
val_accuracy = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        val_loss += outputs.loss.item()
        
        preds = torch.argmax(outputs.logits, dim=-1)
        val_accuracy += get_batch_accuracy(preds, labels)

val_loss /= len(val_loader)
val_accuracy /= len(val_loader)
print(f"Valid - Loss: {val_loss:.4f} Accuracy: {val_accuracy:.4f}")


Valid - Loss: 0.2803 Accuracy: 0.9490
